<a href="https://colab.research.google.com/github/shaunck96/Text-To-Speech-Generation-/blob/main/Customer_Service_Audio_Training_Data_Generation_Using_Text_To_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install --upgrade tensorflow


In [ ]:
from transformers import pipeline
import torch
from datasets import load_dataset
import soundfile as sf
import numpy as np
import os

# Initialize the text-to-speech pipeline
synthesizer = pipeline("text-to-speech", model="microsoft/speecht5_tts")

# Load the dataset for embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# Function to get speaker embedding
def get_speaker_embedding(index):
    return torch.tensor(embeddings_dataset[index]["xvector"]).unsqueeze(0)

# Define speaker embeddings for the customer and the agent
customer_embedding = get_speaker_embedding(7306)  # Example index
agent_embedding = get_speaker_embedding(1000)    # Example index

# Define a function to generate and save conversation audio
def generate_conversation_audio(conversation, topic, synthesizer, customer_embedding, agent_embedding):
    audio_segments = []
    for speaker, text in conversation:
        speaker_embedding = customer_embedding if speaker == "Customer" else agent_embedding
        result = synthesizer(text, forward_params={"speaker_embeddings": speaker_embedding})
        audio_segments.append(result["audio"])
    merged_audio = np.concatenate(audio_segments)
    file_path = os.path.join("generated_conversations", f"{topic.lower().replace(' ', '_')}.wav")
    sf.write(file_path, merged_audio, samplerate=22050)
    print(f"Generated and saved conversation for topic '{topic}' to '{file_path}'")

# Ensure the output directory exists
os.makedirs("generated_conversations", exist_ok=True)

# Example conversation topics and texts
topics_conversations = {
    "Auto-Pay": [
        ("Customer", "Good morning, I've been trying to set up auto-pay for my account, but I seem to be hitting a snag. Can you help me out?"),
        ("Agent", "Absolutely, I'd be happy to assist with that. Could you provide me with your account number so I can pull up your information?"),
        ("Customer", "Sure, it's 123456789. I just want to make sure my bill is paid automatically each month to avoid any late fees."),
        ("Agent", "I understand the convenience that brings. I've got your account details here. Have you tried setting up auto-pay through our online portal or app?"),
        ("Customer", "Yes, I attempted through the website, but it kept giving me an error message when I tried to save my payment information."),
        ("Agent", "I see, it sounds like there might be a glitch on our end. Let's try to set it up together. Do you have your banking information or credit card ready?"),
        ("Customer", "I do. I have my checking account details and also a credit card, whichever works best."),
        ("Agent", "Let's go with the checking account to avoid any possible credit card fees. Please go ahead and give me the routing number followed by your account number."),
        ("Customer", "Okay, the routing number is 12345678, and the account number is 987654321."),
        ("Agent", "Great, let me input that for you..."),
        ("Agent", "And done! Auto-pay is now set up. Your bill will be automatically paid from your checking account on the due date each month."),
        ("Customer", "That was easier than I thought! Can you confirm the date each month my bill will be paid? I want to make sure there are sufficient funds."),
        ("Agent", "Of course, your bill is due on the 15th of each month, so the auto-payment will be processed on that day."),
        ("Customer", "Perfect, thanks for sorting that out. Is there anything else I need to do on my end?"),
        ("Agent", "You're all set! Just ensure there are enough funds in your account before the payment date. You'll receive an email confirmation each month after the payment is processed."),
        ("Customer", "Will do. Thanks for your help today."),
        ("Agent", "You're welcome! If you have any more questions or need further assistance, feel free to contact us. Have a great day!"),
    ],
    "Refund Requests": [
        ("Customer", "Hello, I just noticed on my latest statement that I was charged twice for last month's bill. I need help getting this sorted."),
        ("Agent", "I'm sorry to hear about the double charge. Let's get this resolved for you. Can I have your account number, please?"),
        ("Customer", "Sure, it's 1122334455. The duplicate charge is quite a bit, and I'm hoping to get this refunded as soon as possible."),
        ("Agent", "Thank you for providing your account number. Allow me a moment to review your billing history and identify the duplicate charge."),
        ("Agent", "I've found the duplicate transaction on your account. It appears to have been an error on our part. I'll initiate a refund for the overcharged amount right away."),
        ("Customer", "How long will it take for the refund to process? I need to adjust my budget accordingly."),
        ("Agent", "The refund process usually takes about 5 to 7 business days to be reflected in your bank account. I will also send you a confirmation email with the details of the refund transaction."),
        ("Customer", "Is there a way to prevent this from happening in the future? It's quite inconvenient to have to check for errors like this."),
        ("Agent", "I understand your concern, and I apologize for the inconvenience. We continuously work on improving our billing processes to prevent such errors. However, reviewing your monthly statement can help catch any discrepancies early."),
        ("Customer", "Okay, I'll make sure to do that. Can you confirm the amount being refunded?"),
        ("Agent", "Absolutely. The amount being refunded to your account is $100.50, which is the exact amount of the overcharge."),
        ("Customer", "Thank you for confirming. Will I get any notification once the refund has been processed?"),
        ("Agent", "Yes, you'll receive an email notification once the refund has been processed and another once the funds have been credited back to your account."),
        ("Customer", "Great, that's all I needed to know. Thank you for your help."),
        ("Agent", "You're welcome! If there's anything else I can assist you with or any more questions you have, please don't hesitate to ask. Have a good day!"),
    ],
    "Comparing utility rates": [
        ("Customer", "Hi, I'm considering switching my utility provider and heard you offer competitive rates. How can I compare your rates with my current provider?"),
        ("Agent", "Good morning! I'd be happy to help you with that. The best way to compare rates is by using our online comparison tool available on our website. It allows you to input your current usage and see how our plans stack up against your current provider."),
        ("Customer", "Sounds good. What specific details should I look out for when comparing these rates?"),
        ("Agent", "When comparing rates, it's important to look at the price per kWh for electricity or per therm for gas, depending on what you're comparing. Also, check if the rate is fixed or variable, as this can affect your bill if market prices change."),
        ("Customer", "I see. And are there any additional fees I should be aware of when switching?"),
        ("Agent", "Yes, some plans might have a monthly service fee or a minimum usage requirement. It's also a good idea to check if there's an early termination fee should you decide to switch providers before your contract ends."),
        ("Customer", "Got it. Can you tell me more about the difference between fixed and variable rates?"),
        ("Agent", "Certainly. A fixed rate means the price per kWh or per therm stays the same for the duration of your contract, offering predictability in your bills. A variable rate can change based on market conditions, potentially offering savings when prices are low but also posing a risk if prices increase."),
        ("Customer", "That makes sense. Do you offer any plans that are particularly environmentally friendly?"),
        ("Agent", "Yes, we're proud to offer green energy plans that source electricity from 100% renewable sources like wind and solar. These plans are a great way to reduce your carbon footprint while potentially saving on your energy bill."),
        ("Customer", "Interesting. How can I sign up for a plan once I've made my decision?"),
        ("Agent", "Once you've chosen a plan, you can sign up directly through our website or give us a call, and we'll assist you with the enrollment process. It's quick and easy, and we'll handle the switch for you."),
        ("Customer", "Perfect, I'll take a look at the comparison tool and go from there. Thanks for all the information."),
        ("Agent", "You're very welcome! If you have any more questions or need further assistance as you explore your options, please don't hesitate to reach out. Have a great day and happy comparing!"),
    ],
    "Service transition": [
        ("Customer", "Good morning, I recently started running a business from home and was told I might need to switch my service from residential to commercial. Can you help me with this?"),
        ("Agent", "Certainly, I'd be glad to assist you with transitioning your service to a commercial account. We'll need to review your current usage to ensure we provide you with the best rate plan suited for your business needs."),
        ("Customer", "What information do you need from me to get started?"),
        ("Agent", "To begin, could you provide me with your current account number and some details about your business, such as the type of business, expected energy usage, and if you have any specific requirements?"),
        ("Customer", "Sure, my account number is 987654321. It's a small graphic design studio, so I'm using computers, printers, and a few other machines regularly throughout the day."),
        ("Agent", "Thank you for the information. Based on what you've described, we'll classify your service as a small commercial account. There are different rate plans available that could potentially lower your monthly costs."),
        ("Customer", "Is there a fee associated with changing my account type?"),
        ("Agent", "There's no fee for changing the service type; however, commercial rates are structured differently than residential rates, primarily based on usage and demand. I'll ensure we find a plan that aligns with your business's energy consumption."),
        ("Customer", "How long does the transition process take?"),
        ("Agent", "The transition process can take up to 7 business days. We'll need to conduct a brief audit of your current usage and may need to update your meter to accommodate commercial billing."),
        ("Customer", "Will my service be interrupted during the transition?"),
        ("Agent", "There should be no interruption to your service. If we need to update your meter, we'll schedule a time that's convenient for you, ensuring minimal impact on your business."),
        ("Customer", "That sounds good. How do I officially start the process?"),
        ("Agent", "I'll initiate the transition process on my end and send you an email with a summary of the next steps, including any information we'll need from you and details on potential rate plans. You'll also have a direct line to contact me for any questions during the transition."),
        ("Customer", "Thank you for making this easy. I'll look out for your email and gather the necessary information."),
        ("Agent", "You're welcome! I'm here to make sure this transition is smooth and beneficial for your business. Don't hesitate to reach out if you have any more questions. Have a great day!"),
    ],
    "Porting phone numbers": [
        ("Customer", "Hello there, I'm in the process of switching my phone service to your company and I want to keep my existing number. Can you explain how that works?"),
        ("Agent", "Of course, I'd be happy to help with that. Keeping your current number, or 'porting' as it's called, is quite common. We'll just need some information from your current service to get started."),
        ("Customer", "What kind of information do you need from me to begin the porting process?"),
        ("Agent", "We'll need your current phone number, the account number with your current provider, and the name on the account. It's also helpful to have a recent bill handy as it may contain all the required information."),
        ("Customer", "I have all that information. How long will the porting process take?"),
        ("Agent", "Once we have all the necessary details, porting your number typically takes between 3 to 5 business days. We'll handle the transfer process with your current provider."),
        ("Customer", "Will my service be interrupted during the porting process?"),
        ("Agent", "There might be a brief period, usually just a few hours, where the service switches over. We'll do our best to minimize any disruption during this time."),
        ("Customer", "Is there any cost associated with porting my number?"),
        ("Agent", "No, there's no cost for porting your number to our service. We're happy to provide this as a free service to our new customers."),
        ("Customer", "What happens if I receive calls to my number during the transition?"),
        ("Agent", "Calls to your number will still come through as usual. You might not be able to make outgoing calls during the brief switch-over period, but incoming calls should not be affected."),
        ("Customer", "Alright, that sounds good. What do I need to do next to initiate the porting process?"),
        ("Agent", "I'll start the process on my end right now. I just need to collect the information we discussed. Once I have that, I'll submit the porting request, and we'll take care of the rest."),
        ("Customer", "Great, I'll give you all the details now then."),
        ("Agent", "Perfect, I'm ready when you are. Once we have everything submitted, I'll keep you updated on the progress of your number porting. Welcome to our service, and if you have any other questions, feel free to ask."),
        ("Customer", "Thank you for making this easy. I appreciate your help."),
        ("Agent", "You're welcome! We're here to make sure your switch to our service is as smooth as possible. Have a great day!"),
    ],
    "Service installation": [
        ("Customer", "Hi, I've just moved into a new home and need to set up utility services. Can you walk me through what's involved in the installation process?"),
        ("Agent", "Certainly, I'd be glad to help you get started. First, we'll need to determine the type of services you're looking to install—electric, gas, water, etc. Then, we'll schedule an installation appointment."),
        ("Customer", "I need both electric and gas services. How soon can you schedule the installation?"),
        ("Agent", "We can typically schedule installations within 5 to 7 business days. I'll need your address and some additional details to check our availability in your area."),
        ("Customer", "Okay, my address is 1234 Maple Street. What other details do you need from me?"),
        ("Agent", "Thank you for providing your address. I'll also need to know if your home is already equipped with gas and electric meters or if we need to install new ones."),
        ("Customer", "I believe there are meters already installed, but they were shut off by the previous owner."),
        ("Agent", "Understood. In that case, we'll need to perform a safety inspection of the existing meters and connections before we can reactivate your service. I can schedule an inspector to come out and review everything."),
        ("Customer", "Do I need to be home for the inspection and installation?"),
        ("Agent", "Yes, we require someone to be home to provide access to the meter locations and to ensure that all safety protocols are followed. It's also an opportunity for you to ask any questions about your new service."),
        ("Customer", "Makes sense. Can you give me an idea of what the installation day will look like?"),
        ("Agent", "Of course. On the day of your installation, a technician will arrive within a scheduled window to inspect the meters, ensure they're safe to use, and activate your service. They'll also test the connections to make sure everything is working correctly."),
        ("Customer", "Is there anything I need to do to prepare for the technician's visit?"),
        ("Agent", "Please ensure that the area around your gas and electric meters is clear and accessible. If you have pets, make sure they're secured. Our technician will handle the rest and ensure your service is up and running smoothly."),
        ("Customer", "Great, that all sounds good. Can we go ahead and schedule the installation?"),
        ("Agent", "Absolutely, let me check our schedule for the earliest availability... It looks like we can have a technician out to your home on Monday morning. Does that work for you?"),
        ("Customer", "Monday morning is perfect. Let's book it."),
        ("Agent", "You're all set for Monday morning between 8 AM and 12 PM. I'll send a confirmation email with all the details and a reminder the day before. Welcome to our service, and if there's anything else you need, just let us know."),
        ("Customer", "Thank you for all your help. I'm looking forward to getting everything set up."),
        ("Agent", "You're welcome! We're here to make sure your transition is as smooth as possible. Have a great day!"),
    ],
    "Service relocation": [
        ("Customer", "Hi, I'm moving to a new house soon and need to transfer my utility services. What do I need to do?"),
        ("Agent", "Hello! I'd be happy to assist you with your service relocation. Could you provide me with your current account details and the address of your new home?"),
        ("Customer", "Sure, my account number is 987654321, and the new address is 789 Pine Street."),
        ("Agent", "Great, thank you. When is your move-in date? It's important to schedule the transfer so you have service when you arrive."),
        ("Customer", "My move-in date is the 15th of next month."),
        ("Agent", "Perfect timing. We recommend scheduling the transfer at least two weeks in advance. This gives us time to ensure everything is set up properly at your new address."),
        ("Customer", "Okay, that works. Will my rates be the same at the new address?"),
        ("Agent", "Rates can vary by location due to different local taxes and utility regulations. I'll check the rates for your new address and provide you with the details."),
        ("Customer", "I also have a few smart home devices connected to my current service. Will I be able to use them at my new home?"),
        ("Agent", "Absolutely. We'll make sure your new service supports all your smart devices. When our technician installs the service at your new home, they can help ensure everything is connected correctly."),
        ("Customer", "That's great to hear. How long will the installation take at my new home?"),
        ("Agent", "The installation process typically takes about 2-3 hours, depending on the specifics of your new home and the services you're transferring."),
        ("Customer", "And what about my current service? I'll need it until the day I move."),
        ("Agent", "We'll schedule the disconnection of your current service for the day after your move to ensure you have service right up until you leave. Your new service will be active from the day you move in."),
        ("Customer", "Sounds good. Can we go ahead and schedule everything now?"),
        ("Agent", "Of course. I'll set up the disconnection at your current home for the 16th and start the new service at 789 Pine Street on the 15th. I'll send a confirmation email with all the details and instructions."),
        ("Customer", "Thank you for making this easy. Is there anything else I need to know or do?"),
        ("Agent", "You're all set. Just make sure to clear access to any utility meters or installation points at both homes for our technicians. We'll handle the rest. If you have any questions as your move date approaches, feel free to reach out."),
        ("Customer", "Will do. Thanks for all your help."),
        ("Agent", "You're welcome! We wish you a smooth move and look forward to serving you at your new address. Have a great day!"),
    ],
    "Service contracts": [
        ("Customer", "Hello, I've been considering making some changes to my utility services, but I'm not clear on the terms of my current contract. Can you explain this to me?"),
        ("Agent", "Certainly, I can help with that. Could you provide me with your account number so I can pull up the specific details of your agreement?"),
        ("Customer", "Sure, it's 456789123. I'm particularly interested in understanding the duration of my contract and if there are any penalties for making changes."),
        ("Agent", "Let me quickly review your account. Okay, you're currently on a 12-month fixed-rate plan that started three months ago. This plan does have an early termination fee if you decide to cancel or change to a different plan before the end of the contract period."),
        ("Customer", "I see. How much is the early termination fee?"),
        ("Agent", "The fee for early termination is $150. However, the fee decreases by $12.50 each month you remain on the plan."),
        ("Customer", "What about changes to the service? For example, if I wanted to add more features or upgrade my plan, would there still be a penalty?"),
        ("Agent", "Great question. Upgrading your plan or adding services usually doesn't incur an early termination fee. Fees are typically applied only if you're reducing your level of service or switching providers."),
        ("Customer", "That's good to know. Can you tell me more about the process for upgrading my plan?"),
        ("Agent", "Absolutely. To upgrade, you can either visit our website or call us directly. We'll review your current usage and needs to recommend the best plan options. Upgrading is a simple process and can often be done effective immediately or at your billing cycle's start."),
        ("Customer", "And if I decide to cancel my service entirely, what's the process for that?"),
        ("Agent", "To cancel your service, you'd need to contact us directly. We'd review your account to inform you of any applicable early termination fees and process the cancellation. We always recommend calling at least 30 days before you plan to end service to avoid any overlap in billing."),
        ("Customer", "Okay, that makes sense. I'll need to think about what I want to do. Can I contact you directly if I have more questions?"),
        ("Agent", "Of course, you can contact me or any of our customer service representatives for assistance. We're here to help you make the best decision for your needs."),
        ("Customer", "Thanks for clarifying everything. I appreciate your help."),
        ("Agent", "You're welcome! Don't hesitate to reach out if you have any more questions. Have a great day!"),
    ],
}

# Generate and save audio for each topic
for topic, conversation in topics_conversations.items():
    generate_conversation_audio(conversation, topic, synthesizer, customer_embedding, agent_embedding)


Generated and saved conversation for topic 'Auto-Pay' to 'generated_conversations/auto-pay.wav'
